In [10]:
import plotly.graph_objects as go
from utils import data

big_df = data.get_df('XRPUSDT-1m')
big_df['middle_rolling'] = big_df['close'].rolling(15).mean()
big_df['middle_trend'] = big_df['middle_rolling'] - big_df['middle_rolling'].shift(1)
big_df["drop"] = ((big_df["low"] / big_df["close"] - 1) < -0.01) & (big_df['middle_trend'] < 0)

selection = big_df[big_df['drop']]

In [24]:
sample = selection.sample(1)

min_time = (sample['open_time'] - 1440_0000).min()
max_time = (sample['open_time'] + 1440_0000).min()

df = big_df[(big_df['open_time']>= min_time) & (big_df['open_time']<= max_time)]
df = df.copy()

df['middle_rolling'] = df['close'].rolling(15).mean()
df['low_rolling'] = df['close'].rolling(3).mean()
df['recent_max'] = df['close'].rolling(15).max()

df['short_trend'] = df['low_rolling'] - df['low_rolling'].shift(1)
df['middle_trend'] = df['middle_rolling'] - df['middle_rolling'].shift(1)

df['lows'] = (df['short_trend'] > 0) & (df['middle_trend'] < 0) & ((df['close']/df['recent_max'] - 1) < -0.01)

df['has_recent_buy'] = df['lows'].shift(1).rolling(5).max() == 1

annotate = df[(df['lows']) & (df['has_recent_buy'] == False)][['open_time', 'close', 'recent_max', 'has_recent_buy']]

annotate.head(15)

,open_time,close,recent_max,has_recent_buy
1514902,1616614260000,0.52510,0.53967,False
1514915,1616615040000,0.51312,0.52611,False
1514964,1616617980000,0.51348,0.52406,False
1515075,1616624640000,0.46323,0.51577,False
1515133,1616628120000,0.48082,0.48585,False
1515169,1616630280000,0.48340,0.48928,False
1515208,1616632620000,0.48393,0.49167,False
1515269,1616636280000,0.47920,0.48459,False
1515285,1616637240000,0.47500,0.48273,False
1515294,1616637780000,0.46914,0.47903,False


In [26]:
fig = go.Figure(data=[
    go.Candlestick(x=df['open_time'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close']),
    go.Scatter(x=df['open_time'], y=df['low_rolling'], name='short_trend', hovertext=df['short_trend'] * 100),
    go.Scatter(x=df['open_time'], y=df['middle_rolling'], name='middle_trend', hovertext=df['middle_trend'] * 100),
                      ])

for each in annotate.to_dict('records'):
    fig.add_annotation(
        x=each['open_time'],
        y=each['close'],
        text="SIGNAL",
        showarrow=True,
        arrowhead=2
    )

fig.show()